In [1]:
! pip install pyspark

In [2]:
from pyspark.sql import SparkSession

packages = [
    "com.amazonaws:aws-java-sdk-s3:1.12.95",
    "org.apache.hadoop:hadoop-aws:3.2.0",
    "org.apache.spark:spark-avro_2.12:2.4.4",
    "org.postgresql:postgresql:42.2.18"

]

spark = SparkSession.builder.appName("staging_expenditure_per_cat")\
    .master('spark://172.1.0.2:7077')\
    .config("spark.jars.packages", ",".join(packages))\
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.amazonaws#aws-java-sdk-s3 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-02e7705e-17c2-45ce-8cb1-f5e138b2c35d;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk-s3;1.12.95 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.95 in central
	found com.amazonaws#aws-java-sdk-core;1.12.95 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.12.3 in central
	found com.fasterxml.jackson.core#jackson-annotat

In [3]:
df_user_info = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.1.0.20:5432/wedding") \
    .option("dbtable", "user_info") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_wedding_user = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.1.0.20:5432/wedding") \
    .option("dbtable", "wedding_user") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_wedding_event = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.1.0.20:5432/wedding") \
    .option("dbtable", "wedding_event") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_wedding_budget_list = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.1.0.20:5432/wedding") \
    .option("dbtable", "wedding_budget_list") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_budgets_cat = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.1.0.20:5432/wedding") \
    .option("dbtable", "budgets_cat") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [10]:
import pyspark.sql.functions as F 
all_user_info = df_user_info.join(df_wedding_user,df_wedding_user.user_id == df_user_info.id).join(df_wedding_event,df_wedding_user.wedding_event_id ==df_wedding_event.id)
# df_wedding_days = all_user_info.select("wedding_event_id","wedding_date","created_at").distinct().withColumnRenamed("created_at","wedding_created_at_date")
df_wedding_days = all_user_info.select("wedding_event_id","budget","wedding_date",F.to_date(F.col("created_at"),"yyyy-MM-dd")).distinct().withColumnRenamed("to_date(created_at, yyyy-MM-dd)","wedding_created_at_date")

In [11]:
temp1 = df_wedding_budget_list.join(df_budgets_cat,df_budgets_cat.id == df_wedding_budget_list.budget_cat_id).groupBy("wedding_event_id","budget_cat").count().withColumnRenamed("count","transaction_times")
temp2 = df_wedding_budget_list.join(df_budgets_cat,df_budgets_cat.id == df_wedding_budget_list.budget_cat_id).groupBy("wedding_event_id","budget_cat").sum("expenditure").withColumnRenamed("sum(expenditure)","expenditure_per_cat")
df_expend_perwed_percat = temp1.join(temp2,["wedding_event_id","budget_cat"])
df_expend_perwed_percat

DataFrame[wedding_event_id: int, budget_cat: string, transaction_times: bigint, expenditure_per_cat: bigint]

In [16]:
final_df=df_wedding_days.join(df_expend_perwed_percat,["wedding_event_id"])

In [17]:
RDS_DB="wedding"
RDS_USERNAME="postgres"
RDS_PASSWORD="postgres"
RDS_HOST="final-project-data-warehouse.cit8sojr7959.ap-southeast-1.rds.amazonaws.com"
final_df.write.format('jdbc')\
    .option('url', "jdbc:postgresql://{}/{}".format(RDS_HOST, RDS_DB))\
    .option('dbtable', 'staging_expenditure_per_cat')\
    .option('user', RDS_USERNAME)\
    .option('password', RDS_PASSWORD)\
    .option('driver', 'org.postgresql.Driver')\
    .mode('append')\
    .save()